In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text

In [2]:
data = pd.read_csv('SMSSpamCollection.tsv', sep='\t', names=['target', 'text'])
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data.target.value_counts()

target
ham     4825
spam     747
Name: count, dtype: int64

In [4]:
data['target'] = data['target'].apply(lambda x: 1 if x == 'spam' else 0)

In [5]:
data.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'])

In [7]:
X_train

3883    Short But Cute: "Be a good person, but dont tr...
1662    You aren't coming home between class, right? I...
2050             How much is blackberry bold2 in nigeria.
2741                                      Nothing. Can...
2799    Purity of friendship between two is not about ...
                              ...                        
2301    Nothin comes to my mind. Ü help me buy hanger ...
115     Wa, ur openin sentence very formal... Anyway, ...
77      I like you peoples very much:) but am very shy...
1872    Dont know supports ass and srt i thnk. I think...
81                       K. Did you call me just now ah? 
Name: text, Length: 4179, dtype: object

In [8]:
y_train

3883    0
1662    0
2050    0
2741    0
2799    0
       ..
2301    0
115     0
77      0
1872    0
81      0
Name: target, Length: 4179, dtype: int64

In [11]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\MILAD-CO\AppData\Local\Temp\tfhub_modules\602d30248ff7929470db09f7385fc895e9ceb4c0' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

In [12]:
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

HTTPError: HTTP Error 403: Forbidden

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l1 = tf.keras.layers.Dropout(0.3, name='dropout')(outputs['pooled_output'])
l2 = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l1)

In [19]:
model = tf.keras.Model(inputs=[text_input], outputs=[l2])

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [23]:
model.fit(X_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
131/131 [==============================] - 681s 5s/step - loss: 0.3381 - accuracy: 0.8686
Epoch 2/3
131/131 [==============================] - 870s 7s/step - loss: 0.2499 - accuracy: 0.8890
Epoch 3/3
131/131 [==============================] - 894s 7s/step - loss: 0.2107 - accuracy: 0.9148


In [24]:
y_predicted = model.predict(X_test)
y_predicted

44/44 [==============================] - 179s 4s/step


array([[0.00787523],
       [0.11363889],
       [0.0120696 ],
       ...,
       [0.14111687],
       [0.1730822 ],
       [0.05307255]], dtype=float32)

In [25]:
y_predicted = y_predicted.flatten()
y_predicted

array([0.00787523, 0.11363889, 0.0120696 , ..., 0.14111687, 0.1730822 ,
       0.05307255], dtype=float32)

In [27]:
# MasoudKaviani.ir
# dataset: https://drive.google.com/file/d/1-1hCBHrF1mUvtk7sMlOvzgSgh3T52IMp/view